# Imports and Filter Definitions

In [26]:
import pandas as pd
import numpy as np
import requests
from pprint import pprint
from datetime import datetime
from statistics import mean

# Test single request

In [2]:
# test single address request

URL = "https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x=-70.9404278&" + \
    "y=42.12665739&benchmark=Public_AR_Census2010&vintage=Census2010_Census2010&layers=14" + \
    "&format=json"
  
r = requests.get(url = URL)

data = r.json()

pprint(data)

{'result': {'geographies': {'Census Blocks': [{'AREALAND': 7941,
                                               'AREAWATER': 0,
                                               'BASENAME': '1018',
                                               'BLKGRP': '1',
                                               'BLOCK': '1018',
                                               'CENTLAT': '+42.1270808',
                                               'CENTLON': '-070.9408768',
                                               'COUNTY': '023',
                                               'FUNCSTAT': 'S',
                                               'GEOID': '250235202011018',
                                               'HU100': 0,
                                               'INTPTLAT': '+42.1270808',
                                               'INTPTLON': '-070.9408768',
                                               'LSADC': 'BK',
                                               'LWBLKTYP': 

In [4]:
tract = data['result']['geographies']['Census Blocks'][0]['TRACT']
print(tract)

520201


# Single Request Function Definition

In [10]:
def get_tract_number(longitude, latitude):
    '''
        Makes a REST API request to Census Geocoder API and returns a tract number for a given address.
        Inputs:
            - addr_num = street number, can be a range of street numbers, e.g. 200-400
            - addr_str = street name
            - addr_zip = zip code
            - city = city/town name
        Note: addr_num, addr_str, and addr_zip OR (city/town + state) are required fields! If any are unavailable, then the request fails.
            This is despite the API documentation stating that zip code is an optional field.
    '''
    URL = "https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x=" + str(longitude) \
        + "&y=" + str(latitude) \
        + "&benchmark=Public_AR_Census2010&vintage=Census2010_Census2010&layers=14&format=json"

    response = requests.get(url = URL)
    tract = np.nan
    
    if (response.status_code == 200):
        data = response.json()
        # pprint(data)
        if (len(data['result']['geographies']['Census Blocks']) > 0):
            # if data['result']['addressMatches'][0]['geographies']['Census Blocks'][0]['TRACT']:
            tract = data['result']['geographies']['Census Blocks'][0]['TRACT']
    
    return tract

In [11]:
# test function

print(get_tract_number(-70.9404278, 42.12665739))

520201


# Main program

In [12]:
df = pd.read_csv('../../output csv files/state_land_plus_long_lat.csv', \
                 dtype={'addr_zip': np.string_}) # zip codes need to be imported as string in order to retain leading 0

In [14]:
df.dtypes # check to confirm that longitude and latitude are float by default

index            int64
objectid         int64
mapc_id          int64
muni_id          int64
muni            object
parloc_id       object
poly_typ        object
map_num         object
mappar_id       object
loc_id_cnt     float64
land_value     float64
bldg_value     float64
othr_value     float64
total_value    float64
ls_price       float64
ls_date         object
bldg_area      float64
res_area       float64
luc_1           object
luc_2           object
luc_adj_1      float64
luc_adj_2      float64
num_units      float64
units_est      float64
units_src       object
num_rooms      float64
yr_built       float64
site_addr       object
addr_str        object
addr_num        object
addr_zip        object
owner_name      object
owner_addr      object
owner_city      object
owner_stat      object
owner_zip       object
fy             float64
lot_areaft     float64
far            float64
pct_imperv     float64
pct_bldg       float64
pct_pave       float64
landv_pac      float64
bldgv_psf  

In [16]:
# can ignore this block because we already reset the indices when we did the coordinate conversion

# # reset indices in order to refer to rows by index in for loop below
# # keeps original index values in new column 'index'
# land_parcel_df = land_parcel_df.reset_index()

In [16]:
df['census_tract'] = np.nan

In [17]:
for i in range(len(df)):
    x = df.at[i, 'longitude'] 
    y = df.at[i, 'latitude']

    if (i % 100 == 0):
        print('Step: ', i)
        print("Current Time =", datetime.now().strftime("%H:%M:%S"))

    # check for any blanks or nan values
    if not ((not x or x == np.nan or x == 'nan') \
        or (not y or y == np.nan or y == 'nan')):
            try:
                tract = get_tract_number(x,y)
                df.at[i, 'census_tract'] = tract
            except:
                print('Error at step: ', i)

Step:  0
Current Time = 12:23:09
Step:  100
Current Time = 12:23:34
Step:  200
Current Time = 12:23:59
Step:  300
Current Time = 12:24:25
Step:  400
Current Time = 12:24:50
Step:  500
Current Time = 12:25:16
Step:  600
Current Time = 12:25:41
Step:  700
Current Time = 12:26:07
Step:  800
Current Time = 12:26:33
Step:  900
Current Time = 12:26:58
Step:  1000
Current Time = 12:27:25
Step:  1100
Current Time = 12:27:51
Step:  1200
Current Time = 12:28:18
Step:  1300
Current Time = 12:28:44
Step:  1400
Current Time = 12:29:10
Step:  1500
Current Time = 12:29:40
Step:  1600
Current Time = 12:30:09
Step:  1700
Current Time = 12:30:36
Step:  1800
Current Time = 12:31:01
Step:  1900
Current Time = 12:31:27
Step:  2000
Current Time = 12:31:55
Step:  2100
Current Time = 12:32:21
Step:  2200
Current Time = 12:32:47
Step:  2300
Current Time = 12:33:15
Step:  2400
Current Time = 12:33:42
Step:  2500
Current Time = 12:34:09
Step:  2600
Current Time = 12:34:34
Step:  2700
Current Time = 12:34:59
Step

In [20]:
df.loc[df['census_tract'].notna(), 'census_tract']

0       520201.0
1       520100.0
2       520100.0
3       520202.0
4       520202.0
          ...   
7714    220102.0
7715    220102.0
7716    220102.0
7717    220101.0
7718    220102.0
Name: census_tract, Length: 7719, dtype: float64

# Obtain Median Household Income Estimate by Census Tract for MA

In [21]:
def get_median_hh_income():
    '''
    Returns Pandas DataFrame representation Median Household Income Estimate by Census Tract for MA.
    American Community Survey (ACS) 2018 Census data used.
    Specific table: ACS 2018 5-year detailed table "B19013_001E"
    '''
    URL = "https://api.census.gov/data/2018/acs/acs5?get=B19013_001E&for=tract:*&in=state:25"
  
    response = requests.get(url = URL)
    data = response.json()

    # pprint(data)
    
    median_income_df = pd.DataFrame(data[1:len(data)-1], columns = data[0])
    
    return median_income_df

In [22]:
median_income_df = get_median_hh_income()

In [23]:
median_income_df.head()

,B19013_001E,state,county,tract
0,132750,25,025,060501
1,12759,25,025,061101
2,84083,25,025,070101
3,28851,25,025,070402
4,52676,25,025,071101


In [24]:
# median_income_df.to_csv('ma_med_income_tract.csv',index=False)

# Add Median HH Income Data to land_parcel_df by comparing tract number

In [25]:
median_income_df[median_income_df['tract'] == tract]['B19013_001E']

1039    83110
Name: B19013_001E, dtype: object

In [30]:
# only check for land parcels that we were able to obtain tract numbers for
for i in df[df['census_tract'].notna()].index:
    tract = df.loc[i]['census_tract']
    median_hh_income = median_income_df[median_income_df['tract'] == str(int(tract))]['B19013_001E']
    
    # ACS data is broken down by state > county > tract
    # sometimes tract covered more than 1 county
    # averaged the median incomes
    # print(len(median_hh_income))
    if (len(median_hh_income) > 1):
        median_hh_income = mean([int(m) for m in median_income_df[median_income_df['tract'] == tract]['B19013_001E'].values])
    
    try:
        df.at[i, 'median_hh_income'] = median_hh_income
    except:
        print('Error at index: ', i)

In [32]:
df.loc[df['median_hh_income'].notna(), 'median_hh_income']

0        75089.0
1       103527.0
2       103527.0
3        89978.0
4        89978.0
          ...   
7714     83110.0
7715     83110.0
7716     83110.0
7717     79313.0
7718     83110.0
Name: median_hh_income, Length: 7719, dtype: float64

In [33]:
df.to_csv('state_land_plus_med_income.csv',index=False)